In [6]:

import warnings
warnings.filterwarnings("ignore")

In [7]:
from environments.live_environments import BaseLiveTradingEnv
from neuralforecast.core import NeuralForecast
from Keys import *
import pickle
import numpy as np
from utils import pearl_utils,utils,discord_utils
from Pearl.pearl.utils.instantiations.environments.gym_environment import \
    GymEnvironment
import boto3
from discord import SyncWebhook
from utils import discord_utils
import requests
import matplotlib.pyplot as plt
import pandas as pd
from utils.pandas_reward_functions import log_reward_function,diff_reward_function

In [8]:
channel_url='https://discord.com/api/webhooks/986694946381783102/FOA7nG9ShDcXY95-c3XEKV-Fdek66L9xfbQoKuEuFQkK2P4aFWaZ_fKmzw00j8Oj8Woj'
message='Testing'

In [9]:
from configs import fx_defaults as defaults


In [10]:
forecast_model=NeuralForecast.load(defaults.forecasting_model_path)
# forecast_model=NeuralForecast.load('MultiHeadForecastingModel/')

Seed set to 12
Seed set to 5


In [11]:
forecast_model.scalers_

{'y': <coreforecast.scalers.LocalRobustScaler at 0x1769d4850>,
 'feature_hour_of_day': <coreforecast.scalers.LocalRobustScaler at 0x176927100>,
 'feature_day_of_week': <coreforecast.scalers.LocalRobustScaler at 0x17693ba60>,
 'feature_day_of_month': <coreforecast.scalers.LocalRobustScaler at 0x176938550>,
 'feature_day_of_year': <coreforecast.scalers.LocalRobustScaler at 0x17693a6e0>,
 'feature_BBANDS_BB_UPPER': <coreforecast.scalers.LocalRobustScaler at 0x17678e290>,
 'feature_BBANDS_BB_MIDDLE': <coreforecast.scalers.LocalRobustScaler at 0x17678ff10>,
 'feature_BBANDS_BB_LOWER': <coreforecast.scalers.LocalRobustScaler at 0x17678fdf0>,
 'feature_MACD_MACD': <coreforecast.scalers.LocalRobustScaler at 0x17678ecb0>,
 'feature_MACD_SIGNAL': <coreforecast.scalers.LocalRobustScaler at 0x17678ea10>,
 'feature_14_period_RSI': <coreforecast.scalers.LocalRobustScaler at 0x17678db40>,
 'feature_14_period_STOCH_%K': <coreforecast.scalers.LocalRobustScaler at 0x17678e2c0>}

In [12]:
base_asset=defaults.base_asset
quote_asset=defaults.quote_asset   
test_net=True
time_frame=defaults.time_frame
product_type=defaults.product_type
exchange=defaults.exchange
trade_target=defaults.target_pair
trade_target

'USDJPY'

In [13]:
live_env=BaseLiveTradingEnv(
            api_key=oanda_api_key,
            account_id=oanda_account_id,
            paper=test_net,
            symbol=trade_target,
            time_frame=time_frame,
            product_type=product_type,
            positions=[0,1],
            history_path='Trade_history/trade.db',
            exchange=exchange,
            forecast_model=forecast_model,
            reward_function=diff_reward_function,
            
            discord_webhook='https://discord.com/api/webhooks/986694946381783102/FOA7nG9ShDcXY95-c3XEKV-Fdek66L9xfbQoKuEuFQkK2P4aFWaZ_fKmzw00j8Oj8Woj'

            )

Connected to oanda client
Getting data


100%|██████████| 78/78 [00:00<00:00, 13679.37it/s]
GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


Predicting: |          | 0/? [00:00<?, ?it/s]

GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


Predicting: |          | 0/? [00:00<?, ?it/s]

In [14]:
data=live_env.get_data()

Getting data


100%|██████████| 78/78 [00:00<00:00, 12275.55it/s]
GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


Predicting: |          | 0/? [00:00<?, ?it/s]

GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


Predicting: |          | 0/? [00:00<?, ?it/s]

In [15]:
data

,ds,close,symbol,feature_AutoTFT_H0,feature_AutoTFT_H1,feature_AutoTFT_H2,feature_AutoTFT_H3,feature_AutoTFT_H4,feature_AutoTFT_H5,feature_AutoNBEATS_H0,...,feature_mean_pred_H0,feature_mean_pred_H1,feature_mean_pred_H2,feature_mean_pred_H3,feature_mean_pred_H4,feature_mean_pred_H5,feature_hour_of_day,feature_day_of_week,feature_day_of_month,feature_day_of_year
0,2024-11-22 14:00:00,154.762,USDJPY,154.891785,154.959442,154.936096,154.932999,154.932571,154.932526,154.688339,...,154.79007,154.806946,154.755264,154.662354,154.746613,154.796204,0.108696,0.166667,0.2,0.393151


In [16]:
live_env.prepare_plot_df()

,ds,close,symbol,AutoTFT,AutoNBEATS,mean_pred,cutoff
91,2024-11-20T09:00:00,155.756,USDJPY,NaN,NaN,NaN,NaT
92,2024-11-20T10:00:00,155.752,USDJPY,NaN,NaN,NaN,NaT
93,2024-11-20T11:00:00,155.840,USDJPY,NaN,NaN,NaN,NaT
94,2024-11-20T12:00:00,155.690,USDJPY,NaN,NaN,NaN,NaT
95,2024-11-20T13:00:00,155.732,USDJPY,NaN,NaN,NaN,NaT
96,2024-11-20T14:00:00,155.602,USDJPY,NaN,NaN,NaN,NaT
97,2024-11-20T15:00:00,155.532,USDJPY,NaN,NaN,NaN,NaT
98,2024-11-20T16:00:00,155.384,USDJPY,NaN,NaN,NaN,NaT
99,2024-11-20T17:00:00,155.220,USDJPY,NaN,NaN,NaN,NaT
100,2024-11-20T18:00:00,155.342,USDJPY,NaN,NaN,NaN,NaT


In [17]:
live_env.raw_df

,date_close,open,high,low,close,volume,symbol,ds
0,2024-11-14T14:00:00,156.023,156.034,155.520,155.800,22095,USDJPY,2024-11-14T14:00:00
1,2024-11-14T15:00:00,155.800,156.078,155.652,155.954,22453,USDJPY,2024-11-14T15:00:00
2,2024-11-14T16:00:00,155.956,156.042,155.718,155.972,12337,USDJPY,2024-11-14T16:00:00
3,2024-11-14T17:00:00,155.972,156.044,155.675,155.744,11641,USDJPY,2024-11-14T17:00:00
4,2024-11-14T18:00:00,155.743,155.948,155.698,155.893,9998,USDJPY,2024-11-14T18:00:00
...,...,...,...,...,...,...,...,...
140,2024-11-22T10:00:00,154.500,154.555,154.252,154.328,20136,USDJPY,2024-11-22T10:00:00
141,2024-11-22T11:00:00,154.328,154.470,154.278,154.316,14633,USDJPY,2024-11-22T11:00:00
142,2024-11-22T12:00:00,154.316,154.550,154.294,154.498,14259,USDJPY,2024-11-22T12:00:00
143,2024-11-22T13:00:00,154.498,154.569,154.192,154.276,21414,USDJPY,2024-11-22T13:00:00


In [18]:
history=live_env.load_history()
history

""


In [19]:
live_env.client.get_current_position()

0

In [20]:
live_env.reset()
live_env.step(-1)

Getting data


100%|██████████| 78/78 [00:00<00:00, 12048.16it/s]
GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


Predicting: |          | 0/? [00:00<?, ?it/s]

GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


Predicting: |          | 0/? [00:00<?, ?it/s]

...

Predicting: |          | 0/? [00:00<?, ?it/s]

GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


Predicting: |          | 0/? [00:00<?, ?it/s]

'USDJPY'
'USDJPY'


TypeError: can't multiply sequence by non-int of type 'float'

In [20]:
live_env.client.account()


,balance,pl,unrealizedPL,NAV,marginUsed,marginAvailable,positionValue
USD,9997.2558,-2.4401,0.0000,9997.2558,0.0000,9997.2558,0.0000


In [ ]:
plot_df=live_env.prepare_plot_df()
plot_df.reset_index(drop=True,inplace=True)

In [ ]:
plot_df

In [ ]:
plot_df.dtypes

In [ ]:
fig,axes=plt.subplots(sharex=True, sharey=True, figsize=(10, 5))
time=pd.Timestamp(plot_df['ds'].values[-1]).strftime('%m-%d-%Y %I:%M%p')

axes.plot(plot_df['ds'], plot_df['close'], label='Close Price')
# plot_df['ds'].values[-1]

In [ ]:
live_env.raw_df

In [ ]:
model_name=defaults.model_name
agent_path=f'Agent/pearl_{model_name}_model.pkl'
agent,learning_params=pearl_utils.load_agent_from_study(study_path="sqlite:///pearl_hyper_parameters.sqlite3",
                                        study_name=f'pearl-{model_name}-hp-search',
                                        action_space_dim=2,
                                            observation_space_dim=30,)
agent=pearl_utils.load_agent_weights(agent,weight_path=agent_path)

In [ ]:
from Pearl.pearl.utils.instantiations.environments.gym_environment import \
    GymEnvironment

In [ ]:
live_pearl_env=GymEnvironment(live_env)

In [ ]:
observation,action_space=live_pearl_env.reset()

In [ ]:
agent.observe(observation, action_space)

In [ ]:
action=agent.act(exploit=True)

# action=
action

In [ ]:
live_env.client.account()

In [ ]:
action_result=live_pearl_env.step(int(action))


In [ ]:
len(live_env.client.trade_client.get_fills()['fills'])

In [ ]:
action_result.observation

In [ ]:
df=live_env.df
import pandas as pd

In [ ]:
df['ds_pst']=pd.to_datetime(df['ds'],utc=False)
df[['ds','ds_pst']]

In [ ]:
live_env.client.get_trade_rules()